## Import the using package

In [51]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import requests
import numpy as np
import sklearn
import monpa
import random
import pickle

## The Architecture of the Designed Model using only Bi-GRU 
<img src="https://i.imgur.com/zjsZb0l.png" width = "300" height = "200" alt="design_model" align=center />

In [52]:
class Convolution_1D_Model (nn.Module):
    def __init__(self, total_tag, embeding_size, filter_num_1, filter_size_1, filter_num_2, filter_size_2 , output_size,max_word):
        super(Convolution_1D_Model, self).__init__()
        self.total_tag = total_tag
        self.max_word = max_word
        self.embeding_size = embeding_size
        self.filter_num_1 = filter_num_1
        self.filter_size_1 = filter_size_1
        self.filter_num_2 = filter_num_2
        self.filter_size_2 = filter_size_2
        self.output_size = output_size
        self.Embeding = nn.Embedding(self.total_tag, self.embeding_size)
        self.Conv1d_layer1 = nn.Sequential(nn.Conv1d(in_channels=self.embeding_size, 
                                                            out_channels=self.filter_num_1,
                                                            kernel_size=self.filter_size_1),
                                                         #nn.BatchNorm1d (self.filter_size_1),
                                                         nn.ReLU())
        
        self.Conv1d_layer2 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_1, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2),
                                                         nn.ReLU())
        self.Conv1d_layer3 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_1, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2),
                                                         nn.ReLU())
        self.Conv1d_layer4 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_1, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2),
                                                         nn.ReLU())
        # batch size is  set to 1 , output_size is set to 2
        L = (self.max_word- self.filter_size_1+ 1)- self.filter_size_2+ 1
        self.linear = nn.Linear( self.filter_num_2 * L, self.output_size )
        
    def forward(self, input_data):  
        embed_res = self.Embeding (input_data)
        if len(input_data) < self.max_word:
            for c in range (self.max_word - len(input_data)):
                embed_res = torch.cat((embed_res, torch.zeros(1, self.embeding_size)),dim = 0 )
        embed_res = embed_res.unsqueeze(0)
        #print (' Embedding :::: ' , embed_res.size())
        # let the input dimension be the (N , Cin , L) and then the output with the dimension (N, Cout , Lout)
        conv_res1 = self.Conv1d_layer1 (embed_res.permute(0,2,1))
        #print (' conv_res1 :::: ' , conv_res1.size())
        conv_res2 = self.Conv1d_layer2 (conv_res1)
        #print ('conv_res2 :::: ' , conv_res2.size())
        linear_res = self.linear(conv_res2.view(-1))
        final_res = F.log_softmax(linear_res)
        #print ("final_res :::: " , final_res)
        return final_res

In [53]:
model_v2 = Convolution_1D_Model(total_tag=16, embeding_size=8, filter_num_1=5, filter_size_1=2, filter_num_2=5, filter_size_2=2 ,output_size = 7, max_word=10)
test_vec = torch.LongTensor([1,2,3,4,5,6,7])
model_v2(test_vec)
print ('V2_Model_with_only_convolution1D ::: \n', model_v2)

V2_Model_with_only_convolution1D ::: 
 Convolution_1D_Model(
  (Embeding): Embedding(16, 8)
  (Conv1d_layer1): Sequential(
    (0): Conv1d(8, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (Conv1d_layer2): Sequential(
    (0): Conv1d(5, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (linear): Linear(in_features=40, out_features=7, bias=True)
)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Data Pre-processing

* 將positive_data和negative_data載入存入List
* 將monpa的tags載入dictionary
* 將sentence 的 words 轉成詞性


In [54]:
'''training_data_sentence_list = list()
training_data_tag_list = list()
training_target_sentence_list = list()
dict_idx2tag = dict()
dict_tag2idx = dict()

# Read both the postive data and negative data
with open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:
    for pos_s in rf.readlines():
        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('、').strip('-'))
        training_target_sentence_list.append(1)
with open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:
    for pos_s in rf.readlines():
        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('-'))
        training_target_sentence_list.append(0)
        
print ('training data size : ',len(training_data_sentence_list))
print ('training data target size : ',len(training_target_sentence_list))
tmp_cnt = 0
# Change the words to tags, because we need to use the sequential tags for training.
for sentence in training_data_sentence_list:
    tmp_cnt += 1
    #print (tmp_cnt)
    tmp = monpa.pseg(sentence)
    tmp_list = list()
    for item in tmp:
        tmp_list.append(item[1])
    training_data_tag_list.append(tmp_list)
    
print (len(training_data_tag_list))

# Read the tags of the monpa
with open ('./monpa_tag.txt' , 'r' , encoding = 'utf-8') as rf:
    cnt = 0
    for tag in rf.readlines():
        dict_idx2tag[cnt] = tag.strip('\n')
        dict_tag2idx[tag.strip('\n')] = cnt
        cnt += 1  

print ('The dictionary of idxs 2 tags : ', dict_idx2tag)
print ('The dictionary of tags 2 idxs : ' , dict_tag2idx)

shuffle_zip = list(zip(training_data_tag_list, training_target_sentence_list))
random.shuffle(shuffle_zip)
training_data_tag_list[:], training_target_sentence_list[:] = zip(*shuffle_zip)
print (training_data_tag_list, training_target_sentence_list)
'''


"training_data_sentence_list = list()\ntraining_data_tag_list = list()\ntraining_target_sentence_list = list()\ndict_idx2tag = dict()\ndict_tag2idx = dict()\n\n# Read both the postive data and negative data\nwith open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:\n    for pos_s in rf.readlines():\n        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('、').strip('-'))\n        training_target_sentence_list.append(1)\nwith open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:\n    for pos_s in rf.readlines():\n        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('-'))\n        training_target_sentence_list.append(0)\n        \nprint ('training data size : ',len(training_data_sentence_list))\nprint ('training data target size : ',len(training_target_sen

## Save the training data list

In [55]:
'''f = open ('../pickle/training_tags.pkl', 'wb')
pickle.dump(training_data_tag_list, f)
f.close()
f = open ('../pickle/target.pkl', 'wb')
pickle.dump(training_target_sentence_list, f)
f.close()'''

"f = open ('../pickle/training_tags.pkl', 'wb')\npickle.dump(training_data_tag_list, f)\nf.close()\nf = open ('../pickle/target.pkl', 'wb')\npickle.dump(training_target_sentence_list, f)\nf.close()"

In [56]:
f = open ('../pickle/training_tags.pkl', 'rb')
a = pickle.load(f)
f.close()
f = open ('../pickle/target.pkl', 'rb')
b = pickle.load(f)
f.close()

In [ ]:
dict_idx2tag = dict()
dict_tag2idx = dict()
with open ('./monpa_tag.txt' , 'r' , encoding = 'utf-8') as rf:
    cnt = 0
    for tag in rf.readlines():
        dict_idx2tag[cnt] = tag.strip('\n')
        dict_tag2idx[tag.strip('\n')] = cnt
        cnt += 1  
training_data_tag_list = a
training_target_sentence_list = b

## Hyperparamters_v2

In [70]:
epoch_v2 = 100
learning_rate_v2 = 0.01
input_size_v2 = len(dict_idx2tag.keys())
gru_hidden_size_v2 = 8
embedding_size_v2 = 16
output_size_v2 = 2
gru_layer_v2 = 1
filter_num_1_v2 = 3
filter_size_1_v2 = 3
filter_num_2_v2 = 3
filter_size_2_v2 = 2
max_word_v2 = 20

## 定義訓練function

In [73]:
def train_v2():
    model_v2 = Convolution_1D_Model(total_tag= input_size_v2, 
                                    embeding_size= embedding_size_v2, 
                                    filter_num_1= filter_num_1_v2, 
                                    filter_size_1= filter_size_1_v2, 
                                    filter_num_2= filter_num_2_v2, 
                                    filter_size_2=filter_size_2_v2 ,
                                    output_size = output_size_v2, 
                                    max_word= max_word_v2 )
    optimizer_v2 = torch.optim.SGD(model_v2.parameters(), lr=learning_rate_v2)
    loss_function_v2 = nn.CrossEntropyLoss()
    
    for epoch in range(epoch_v2):
        sub_epoch = 0
        total_loss = 0
        for seq_tag_idx in range(len(training_data_tag_list)):
            if len(training_data_tag_list[seq_tag_idx]) > max_word_v2:
                continue
            sub_epoch += 1
            input_data = torch.LongTensor([dict_tag2idx[t] for t in training_data_tag_list[seq_tag_idx]])
            target_data = torch.LongTensor([training_target_sentence_list[seq_tag_idx]])
            out = model_v2(input_data)
            out = out.unsqueeze(0)
            loss = loss_function_v2(out, target_data)
            total_loss += loss.item()
            optimizer_v2.zero_grad()
            loss.backward()
            optimizer_v2.step()
            '''
            if sub_epoch % 100 == 0:
                print ('Epoch '+  str(epoch) + ' '+ str(sub_epoch)+'/' + str(len(training_data_tag_list)) + '  Loss : ' + str(total_loss))
            '''
        print ('Epoch ' + str(epoch) + ' Loss : ' + str (total_loss) + ' total_train_data : ' + str(sub_epoch) )
        torch.save(model_v2, '../pickle/model_v2.pt')

In [74]:
train_v2()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 Loss : 4876.927813082933 total_train_data : 7014


/anaconda3/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Convolution_1D_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 Loss : 4870.383149147034 total_train_data : 7014
Epoch 2 Loss : 4870.043609023094 total_train_data : 7014
Epoch 3 Loss : 4869.911008358002 total_train_data : 7014
Epoch 4 Loss : 4869.86924213171 total_train_data : 7014
Epoch 5 Loss : 4869.841605484486 total_train_data : 7014
Epoch 6 Loss : 4869.816461861134 total_train_data : 7014
Epoch 7 Loss : 4869.8019688129425 total_train_data : 7014
Epoch 8 Loss : 4869.797327756882 total_train_data : 7014
Epoch 9 Loss : 4869.795873999596 total_train_data : 7014
Epoch 10 Loss : 4869.793142020702 total_train_data : 7014
Epoch 11 Loss : 4869.787251055241 total_train_data : 7014
Epoch 12 Loss : 4869.783266484737 total_train_data : 7014
Epoch 13 Loss : 4869.786013841629 total_train_data : 7014
Epoch 14 Loss : 4869.7828823924065 total_train_data : 7014
Epoch 15 Loss : 4869.785557150841 total_train_data : 7014
Epoch 16 Loss : 4869.781072199345 total_train_data : 7014
Epoch 17 Loss : 4869.781697392464 total_train_data : 7014
Epoch 18 Loss : 4869.7